In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

In [2]:
# Store Part I results into DataFrame
# Load the csv exported in Part I to a DataFrame

cities_path = "../WeatherPy/cities_weather.csv"

# Read School and Student Data File and store into Pandas DataFrames
cities_data = pd.read_csv(cities_path)

#Create the dataframe
cities_data_df = pd.DataFrame(cities_data)
cities_data.head()

,Cities,Countries,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness,Wind Speed (mph),Date
0,Bakel,SN,14.90,-12.46,72.5,20,80,10.5,02-09-2021
1,Jamestown,SH,-15.94,-5.72,68.4,88,54,15.0,02-09-2021
2,Nikolskoye,RU,59.70,30.79,5.0,85,90,2.2,02-09-2021
3,Arraial Do Cabo,BR,-22.97,-42.03,72.4,73,100,2.1,02-09-2021
4,Port Hardy,CA,50.70,-127.42,37.4,74,1,8.0,02-09-2021


In [3]:
#Humidity Heatmap

#Configure gmaps
gmaps.configure(api_key=gkey)

#Use the Lat and Lng as locations and Humidity as the weight.
locations = cities_data_df[["Latitude", "Longitude"]]
weights = cities_data_df["Humidity (%)"]

#Create map
figure_layout = {'width': '800px',
                 'height': '600px',
                 'border': '1px solid black',
                 'padding': '1px',
                 'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=figure_layout, center=(0,0), zoom_level=1)

In [4]:
#Add Heatmap layer to map.
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [5]:
#Create new DataFrame fitting weather criteria
#Narrow down the cities to fit weather conditions.

#Max temp between 75 and 90 degrees F.
#Wind speed between 1 and 15 mph.
#Cloudiness between 10 and 50%

for index, rows in cities_data_df.iterrows():
    if cities_data_df.loc[index, "Max Temperature (F)"] < 90:
        pass
    else:
        cities_data_df.loc[index, "Cities"] = np.nan
        print("skip")
        continue
    if cities_data_df.loc[index, "Max Temperature (F)"] >= 75:
            pass
    else:
        cities_data_df.loc[index, "Cities"] = np.nan
        print("skip")
        continue 
    if cities_data_df.loc[index, "Wind Speed (mph)"] <= 15:
        pass
    else:
        cities_data_df.loc[index, "Cities"] = np.nan
        print("skip")
        continue
    if cities_data_df.loc[index, "Cloudiness"] <= 40:
        pass
    else:
        cities_data_df.loc[index, "Cities"] = np.nan
        print("skip")
        continue
    if cities_data_df.loc[index, "Cloudiness"] >= 5:
        pass
    else:
        cities_data_df.loc[index, "Cities"] = np.nan
        print("skip")
        continue
        
cities_data_df.head()        

skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip


,Cities,Countries,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness,Wind Speed (mph),Date
0,NaN,SN,14.90,-12.46,72.5,20,80,10.5,02-09-2021
1,NaN,SH,-15.94,-5.72,68.4,88,54,15.0,02-09-2021
2,NaN,RU,59.70,30.79,5.0,85,90,2.2,02-09-2021
3,NaN,BR,-22.97,-42.03,72.4,73,100,2.1,02-09-2021
4,NaN,CA,50.70,-127.42,37.4,74,1,8.0,02-09-2021


In [9]:
# Drop any rows will null values.

cities_data_df.dropna(inplace=True)
cities_data_df.reset_index(drop=True, inplace=True)
vacation_df = cities_data_df[["Cities", "Countries", "Latitude", "Longitude"]].copy()
vacation_df

,Cities,Countries,Latitude,Longitude
0,The Valley,AI,18.22,-63.06
1,Olinda,BR,-8.01,-34.86
2,Moree,AU,-29.47,149.85


In [10]:
# Build out new columns of dataframe to add info to

vacation_df["Hotel"] = ""
vacation_df["Address"] = ""
vacation_df["Hotel Lat"] = ""
vacation_df["Hotel Lng"] = ""
vacation_df

,Cities,Countries,Latitude,Longitude,Hotel,Address,Hotel Lat,Hotel Lng
0,The Valley,AI,18.22,-63.06,,,,
1,Olinda,BR,-8.01,-34.86,,,,
2,Moree,AU,-29.47,149.85,,,,


In [32]:
# Use Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

params = {
    "location": "location",
    "rankby": "distance",
    "type": "lodging",
    "keyword": "hotel",
    "key": gkey}

# Use the lat/lng we recovered to identify airports
for index, row in vacation_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel = requests.get(base_url, params=params)

    # print the name_address url, avoid doing for public github repos in order to avoid exposing key
    # print(hotel.url)

    # convert to json
    hotel = hotel.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        vacation_df.loc[index, "Hotel"] = hotel["results"][0]["name"]
        vacation_df.loc[index, "Address"] = hotel["results"][0]["vicinity"]
        vacation_df.loc[index, "hotel_lat"] = hotel["results"][0]["geometry"]["location"]["lat"]
        vacation_df.loc[index, "hotel_lng"] = hotel["results"][0]["geometry"]["location"]["lng"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
vacation_df

,Cities,Countries,Latitude,Longitude,Hotel,Address,Hotel Lat,Hotel Lng,hotel_lat,hotel_lng
0,The Valley,AI,18.22,-63.06,Fountain Residences Anguilla,Fountain Rd Shoal Bay,18.2515,-63.0383,18.251533,-63.038296
1,Olinda,BR,-8.01,-34.86,Pousada do Amparo,"R. do Amparo, 199 - Amparo, Olinda",-8.01297,-34.8533,-8.012968,-34.853305
2,Moree,AU,-29.47,149.85,Mehi River Van Park,"28 Oak St, Moree",-29.471,149.851,-29.471048,149.850650


In [35]:
# Reduce df to only necessary info
vacation_reduced = vacation_df[["Cities", "Countries", "Hotel", "hotel_lat", "hotel_lng"]].copy()
vacation_reduced

,Cities,Countries,Hotel,hotel_lat,hotel_lng
0,The Valley,AI,Fountain Residences Anguilla,18.251533,-63.038296
1,Olinda,BR,Pousada do Amparo,-8.012968,-34.853305
2,Moree,AU,Mehi River Van Park,-29.471048,149.850650


In [37]:
vacation_reduced["Coordinates"] = list(zip(vacation_reduced.hotel_lat, vacation_reduced.hotel_lng))
vacation_reduced

,Cities,Countries,Hotel,hotel_lat,hotel_lng,Coordinates
0,The Valley,AI,Fountain Residences Anguilla,18.251533,-63.038296,"(18.251533, -63.038296)"
1,Olinda,BR,Pousada do Amparo,-8.012968,-34.853305,"(-8.012968299999999, -34.8533049)"
2,Moree,AU,Mehi River Van Park,-29.471048,149.850650,"(-29.4710475, 149.8506497)"


In [38]:
# Make dictionaries
hotels = vacation_reduced.to_dict(orient="records")
hotels

[{'Cities': 'The Valley',
  'Countries': 'AI',
  'Hotel': 'Fountain Residences Anguilla',
  'hotel_lat': 18.251533,
  'hotel_lng': -63.038296,
  'Coordinates': (18.251533, -63.038296)},
 {'Cities': 'Olinda',
  'Countries': 'BR',
  'Hotel': 'Pousada do Amparo',
  'hotel_lat': -8.012968299999999,
  'hotel_lng': -34.8533049,
  'Coordinates': (-8.012968299999999, -34.8533049)},
 {'Cities': 'Moree',
  'Countries': 'AU',
  'Hotel': 'Mehi River Van Park',
  'hotel_lat': -29.4710475,
  'hotel_lng': 149.8506497,
  'Coordinates': (-29.4710475, 149.8506497)}]

In [42]:
hotel_locations = [hotel["Coordinates"] for hotel in hotels]
print(hotel_locations)

[(18.251533, -63.038296), (-8.012968299999999, -34.8533049), (-29.4710475, 149.8506497)]


In [44]:
info_box_template = """
<dl>
<dt>Hotel Name:</dt><dd>{Hotel}</dd>
<dt>City:</dt><dd>{Cities}</dd>
<dt>Country:</dt><dd>{Countries}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**hotel) for hotel in hotels]

markers = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…